# Supervised Learning: Clock Drawing Image Classification with Convolutional Neural Networks
### Stacey Beck and Ian Byrne

- Split data into sets of Training (x = image arrays ; y = labels), Test (~10% image arrays), and Validation (~10% of the Training). 
- Build CNN using Pytorch for Training and Test:
  - Specify CUDA
  - 2D convolution, Normalization (for faster training), Non-linear Activation Function (ex. RELU), Max Pooling (downsampling to reduce learned parameters).
  - Define Layers 
  - Build Forward and backward pass
  - Define optimizer (due to many - deep - nodes) ex) ADAM
  - Calculate Loss (BCE)
  - Calculate Accuracy, Precision, Recall (Confusion Matrix)
  - Plot ROC and print Confusion Matrix



In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

In [ ]:
# General Libraries
import pandas as pd
import numpy as np
import ast
import logging
import os
import tempfile
import glob

# Pytroch Libraries
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
from torch.nn import Linear, ReLU, CrossEntropyLoss, Sequential, Conv2d, MaxPool2d, Module, Softmax, BatchNorm2d, Dropout
from torch.optim import Adam, SGD
from torch.optim import lr_scheduler
from torch.optim.lr_scheduler import ReduceLROnPlateau
from torch.utils.data import Dataset, DataLoader
import torchvision.models as models
import torchmetrics

# To Evaluate model
from tqdm import tqdm
import torchmetrics
from torchmetrics import ConfusionMatrix
from sklearn.metrics import multilabel_confusion_matrix
from sklearn.metrics import plot_confusion_matrix
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

# To visualize model
import matplotlib.pyplot as plt
import seaborn as sns
from PIL import Image
from skimage.io import imread

# To split the data
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import label_binarize
from sklearn.metrics import precision_recall_curve
from sklearn.preprocessing import LabelBinarizer
from sklearn.metrics import roc_curve, auc, roc_auc_score


# Build CNN Model using Pytorch


### Building and Training
Architecture choices influenced from: 

https://www.analyticsvidhya.com/blog/2018/12/guide-convolutional-neural-network-cnn/

https://medium.datadriveninvestor.com/five-powerful-cnn-architectures-b939c9ddd57b

https://towardsdatascience.com/how-does-sparse-convolution-work-3257a0a8fd1

In [ ]:
epochs = 2
train_batch_size = 8
val_batch = 4
test_batch = 1
learning_rate = 1e-3
kernel_size = 3
stride = 1
padding = 1 #2*floor(3/2)
weight_decay = 1e-5
# Define file extension to use for new data saves
extension_ = "res_netfp"
# Normalize data if rgb and set rgb_val to True to convert 
normalize_ = 'True'
# Define which round to get data from
rnd = 7
# Define model extensions for naming file (which model do we want to train on)
model_ext = "3"
# m = 'First model'
# m = 'pre-trained'
#m = 'pre-trained-res'
m = 'resnet'

In [ ]:
def open_dict(path):
    """Takes in path and opens file"""

    cust_file = open(path, "r")
    contents = cust_file.read()
    dictionary = ast.literal_eval(contents)
    cust_file.close()

    return dictionary


dictionarytr = open_dict(
    "/content/gdrive/MyDrive/Data/Dictionaries/score_dicts/tr_scor_dict_bal.txt"
)
dictionaryv = open_dict(
    "/content/gdrive/MyDrive/Data/Dictionaries/score_dicts/val_scor_dict_nobal.txt"
)
dictionaryts = open_dict(
    "/content/gdrive/MyDrive/Data/Dictionaries/score_dicts/tst_scor_dict_nobal.txt"
)

In [ ]:
for id, val in dictionarytr.items():
  print("Round {} length is {}".format(str(id), str(len(val))))

In [ ]:
class ResizedClocks:
    # Resized clock drawing dataset

    def __init__(self, round, round_labels, rgb=None, transform=None):

        # Args:
        # round (int): Round to grab images from.
        # values (list of tuples): Corresponding values for the round.

        self.round = round
        self.vals = round_labels
        self.base_path = '/content/gdrive/MyDrive/Data/Nhats Dataset/NHATS_R11_ClockDrawings_V2'
        self.transform = transform
        self.rgb = rgb

    def __len__(self):
        return len(self.vals)

    def __getitem__(self, idx):
        spid = self.vals[idx][0]
        label = torch.tensor(int(self.vals[idx][1]))
        
        # Try different file extensions
        possible_files = [
            f"{self.base_path}/{spid}.tif",
            f"{self.base_path}/{spid}.jpg",
            f"{self.base_path}/{spid}.png",
            f"{self.base_path}/{spid}.jpeg"
        ]
        
        image_path = None
        for file_path in possible_files:
            if os.path.exists(file_path):
                image_path = file_path
                break
        
        if image_path is None:
            return None  # File not found

        try:
            im = Image.open(image_path)

            if self.rgb:
                gray = im.convert('RGB')
            else:
                gray = im.convert('1')

            resized = gray.resize((160, 207))
            im_arr = np.float32(np.array(resized))

            if self.transform:
                im_arr = self.transform(im_arr)

            return im_arr, label

        except Exception as e:
            print(f"Error loading image {image_path}: {e}")
            return None


In [ ]:
def collate_fn(batch):
  """From pytorch - way to bypass corrupt or non-existent data"""
  batch = list(filter(lambda x: x is not None, batch))
  return torch.utils.data.dataloader.default_collate(batch)

In [ ]:
# initialize transformation: data to tensor and normalize
# Could probably resize using torch.transforms
if normalize_ == "True":
    processes = transforms.Compose(
        [
            transforms.ToTensor(),
            transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]),
        ]
    )
    rgb_val = "True"
else:
    processes = transforms.ToTensor()
    rgb_val = None
    

In [ ]:
train_set = ResizedClocks(rnd, dictionarytr[rnd], transform = processes, rgb = rgb_val)
val_set = ResizedClocks(rnd, dictionaryv[rnd], transform = processes, rgb = rgb_val)
test_set = ResizedClocks(rnd, dictionaryts[rnd], transform = processes, rgb = rgb_val)

In [ ]:
# Define Dataloaders for the network
train_loader = torch.utils.data.DataLoader(train_set, batch_size = train_batch_size, shuffle = True, num_workers = 6, collate_fn=collate_fn) 
validate_loader = torch.utils.data.DataLoader(val_set, batch_size = val_batch, shuffle = True, num_workers = 6, collate_fn=collate_fn) #64, 8,1

In [ ]:
# For round 10, there are some corrupt data that when batched at size 1 is not taken
# care of by the collate function, but Nonechucks library skips the missing data and 
# moves on, replacing that missing data index with the next piece of data
# could probably just use this in place of collate for all the loading
if rnd == 10:
  !pip install nonechucks
  import nonechucks as nc
  test_set_safe = nc.SafeDataset(test_set)
  test_loader = torch.utils.data.DataLoader(test_set_safe, batch_size = test_batch, shuffle = False)
else:
  test_loader = torch.utils.data.DataLoader(test_set, batch_size = test_batch, shuffle = False, collate_fn=collate_fn)

In [ ]:
def set_model(m, model_ext, device):

    if torch.cuda.is_available():
        print("First Model training on GPU")

        if m == "First model":
            # Create model object
            model = ConvNet()
            model = model.to(device)  # (float).cuda()

        elif m == "pre-trained":
            mPATH = "/content/gdrive/MyDrive/Colab Notebooks/Models/cnn_512_662.model{}".format(
                model_ext
            )
            model = ConvNet()
            model.load_state_dict(torch.load(mPATH))
            model.to(device)
            print("New Model{} training on GPU".format(model_ext))

        elif m == "resnet":
            model = models.resnet50(pretrained=True)
            num_ftrs = model.fc.in_features
            model.fc = nn.Linear(num_ftrs, 6)
            model = model.to(device)
            print("RESNET Model training on GPU")

        elif m == "pre-trained-res":
            mPATH = "/content/gdrive/MyDrive/Colab Notebooks/Models/cnn_512_662.model{}".format(
                model_ext
            )
            model = models.resnet50()
            num_ftrs = model.fc.in_features
            model.fc = nn.Linear(num_ftrs, 6)
            model.load_state_dict(torch.load(mPATH))
            model.to(device)
            print("New Model{} training on GPU".format(model_ext))

    else:
        print("CUDA is not available. Turn on GPU")

    return model

In [ ]:
def accuracy(y_pred, y_test):
  # Calculating model accuracy at each epoch 
  y_pred_softmax = torch.log_softmax(y_pred, dim = 1)
  _, y_pred_prob = torch.max(y_pred_softmax, dim = 1)
  correct_pred = (y_pred_prob == y_test).float()
  acc = correct_pred.sum() / len(correct_pred)
  acc = torch.round(acc * 100)

  return acc

In [ ]:
class ConvNet(nn.Module):
    def __init__(self):
        super(ConvNet, self).__init__()

        # without considering batch size: Input shape : (None,368, 284, 1) , parameters: (3*3*1*16+16) = 160
        self.conv1 = nn.Conv2d(
            in_channels=1,
            out_channels=16,  # one input channel gray scale, 16 filters out
            kernel_size=3,
            stride=1,
            padding=1,
        )  # Out:(None,386, 284, 16). ### TRY kernel 7x7 padding 3
        self.conv2 = nn.Conv2d(
            in_channels=16, out_channels=32, kernel_size=3, stride=1, padding=1
        )  # params: (3*3*16*32+32) = 4640
        self.pool1 = nn.MaxPool2d(2, 2)  # Out: (None, 184, 142, 32)
        self.bn1 = nn.BatchNorm2d(32)

        self.conv3 = nn.Conv2d(
            in_channels=32, out_channels=64, kernel_size=3, stride=1, padding=1
        )  # params: (3*3*16*32+32) = 4640
        self.conv4 = nn.Conv2d(
            in_channels=64, out_channels=64, kernel_size=3, stride=1, padding=1
        )  # params: (3*3*32*32+32) = 9248
        self.pool2 = nn.MaxPool2d(2, 2)  # Output shape = (None, 92, 71, 64)
        self.bn2 = nn.BatchNorm2d(64)

        # self.conv5 = nn.Conv2d(in_channels = 64, out_channels = 128,
        # kernel_size = 3, stride = 1, padding = 1) # params: (3*3*32*32+32) = 9248
        self.conv6 = nn.Conv2d(
            in_channels=64, out_channels=128, kernel_size=3, stride=1, padding=1
        )  # params: (3*3*32*32+32) = 9248
        self.pool3 = nn.MaxPool2d(2, 2)  # Output shape = (None, 46, 35, 128)
        self.bn3 = nn.BatchNorm2d(128)
        self.do2 = nn.Dropout(0.3)

        # Fully connected layer
        self.fc1 = nn.Linear(
            128 * 64 * 82, 60
        )  # most recent original size of: 512, 662 -->64 x 82
        self.do3 = nn.Dropout(0.4)  # 40 % probability
        # self.fc3 = nn.Linear(60, 30)
        self.fc2 = nn.Linear(60, 6)  # left with 3 for the three classes

    def forward(self, x):
        x = self.bn1(self.pool1(F.relu(self.conv2(F.relu(self.conv1(x))))))
        x = self.bn2(self.pool2(F.relu(self.conv4(F.relu(self.conv3(x))))))
        # x = self.bn3(self.pool3(F.relu(self.conv6(F.relu(self.conv5(x))))))
        x = self.bn3(self.pool3(F.relu(self.conv6((x)))))
        x = self.do2(x)
        x = x.view(x.size(0), 128 * 64 * 82)
        x = F.relu(self.fc1(x))
        x = self.do3(x)
        x = self.fc2(x)
        return x


In [ ]:
def train_val_model(epochs):
    for epoch in range(1, epochs + 1):

        train_epoch_loss = 0
        train_epoch_acc = 0

        # set model in training mode
        model.train()
        print("\nEpoch$ : %d" % epoch)
        for x_train_batch, y_train_batch in tqdm(train_loader):
            x_train_batch = x_train_batch.to(
                device
            )  # (float).to(device) # for GPU support
            y_train_batch = y_train_batch.to(device)

            # print(x_train_batch.shape)

            # sets gradients to 0 to prevent interference with previous epoch
            optimizer.zero_grad()

            # Forward pass through NN
            y_train_pred = model(x_train_batch)  # .to(float)
            train_loss = criterion(y_train_pred, y_train_batch)
            train_acc = accuracy(y_train_pred, y_train_batch)

            # Backward pass, updating weights
            train_loss.backward()
            optimizer.step()

            # Statistics
            train_epoch_loss += train_loss.item()
            train_epoch_acc += train_acc.item()

        with torch.set_grad_enabled(False):
            val_epoch_loss = 0
            val_epoch_acc = 0

            model.eval()
            for x_val_batch, y_val_batch in tqdm(validate_loader):

                x_val_batch = x_val_batch.to(device)  # .to(float)
                y_val_batch = y_val_batch.to(device)

                # Forward pass
                y_val_pred = model(x_val_batch)  # .to(float)
                val_loss = criterion(y_val_pred, y_val_batch)
                val_acc = accuracy(y_val_pred, y_val_batch)

                val_epoch_loss += val_loss.item()
                val_epoch_acc += val_acc.item()

        # Prevent plateauing validation loss
        # scheduler.step(val_epoch_loss/len(validate_loader))

        loss_stats["train"].append(train_epoch_loss / len(train_loader))
        loss_stats["val"].append(val_epoch_loss / len(validate_loader))
        accuracy_stats["train"].append(train_epoch_acc / len(train_loader))
        accuracy_stats["val"].append(val_epoch_acc / len(validate_loader))

        print(
            f"Epoch {epoch+0:03}: Train Loss: {train_epoch_loss/len(train_loader):.5f} | Val Loss: {val_epoch_loss/len(validate_loader):.5f}"
        )
        print(
            f"Train Acc: {train_epoch_acc/len(train_loader):.3f} | Val Acc: {val_epoch_acc/len(validate_loader):.3f}"
        )



In [ ]:
 # Set to GPU
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

In [ ]:
# Get model
model = set_model(m, model_ext, device)

# Loss function
criterion = nn.CrossEntropyLoss(reduction="mean")

optimizer = torch.optim.Adam(model.parameters(), lr = learning_rate)#, momentum = 0.9) #or ADAM/ momentum
# optimizer = torch.optim.SGD(
#     model.parameters(), lr=learning_rate, weight_decay=weight_decay
# )

scheduler = lr_scheduler.StepLR(optimizer, step_size = 4, gamma=0.1)
# scheduler = ReduceLROnPlateau(optimizer, "min", patience=4)

accuracy_stats = {"train": [], "val": []}

loss_stats = {"train": [], "val": []}

In [ ]:
train_val_model(15)

In [ ]:
# added learning rate decay after 3rd epoch
# 1st: round 1
# 2nd: round 5
# 3rd: round 6 best so far
# 4th: round 7
# 5th (model) saved training on 9
# 6th: round 2
# 7th round 8
# 8th round 3
# 9th round 4
# 10th round 10

# Visualize the Training and Validation


In [ ]:
# Create dataframes
train_val_acc_df = pd.DataFrame.from_dict(accuracy_stats).reset_index().melt(id_vars=['index']).rename(columns={"index":"epochs"})
train_val_loss_df = pd.DataFrame.from_dict(loss_stats).reset_index().melt(id_vars=['index']).rename(columns={"index":"epochs"})

# Create directories if they don't exist
os.makedirs('/content/gdrive/MyDrive/Colab_Notebooks/acc_loss', exist_ok=True)
os.makedirs('/content/gdrive/MyDrive/Colab_Notebooks/model_charts', exist_ok=True)

train_val_acc_df.to_csv('/content/gdrive/MyDrive/Colab_Notebooks/acc_loss/acc{}.csv'.format(extension_), index = False)
train_val_loss_df.to_csv('/content/gdrive/MyDrive/Colab_Notebooks/acc_loss/loss{}.csv'.format(extension_), index = False)
# Plot the dataframes
fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(20,7))
sns.lineplot(data=train_val_acc_df, x = "epochs", y="value", hue="variable",  ax=axes[0]).set_title('Train-Val Accuracy/Epoch')
sns.lineplot(data=train_val_loss_df, x = "epochs", y="value", hue="variable", ax=axes[1]).set_title('Train-Val Loss/Epoch')
fig.savefig('/content/gdrive/MyDrive/Colab_Notebooks/model_charts/acc_loss{}.png'.format(extension_))

# Evaluate the model using Test Set

In [ ]:
# Calculate performance
y_test = torch.tensor([])
test_acc = torchmetrics.Accuracy()

with torch.set_grad_enabled(False):
  model.eval()
  #model.to(float)
  for batches in tqdm(test_loader):
    x_test, y_test = batches
    x_test = x_test.to(device)
    y_test = y_test.to(device)
    y_pred = model(x_test)
    test_acc(y_pred.cpu(), y_test.cpu())
    total_test_acc = test_acc.compute()
  print('test acc: ', total_test_acc)
  test_acc.reset()
   

## Create Confusion Matrix

In [ ]:
all_pred = []
all_preds = torch.tensor([])
y_test = torch.tensor([])
with torch.set_grad_enabled(False):
  model.eval()
  for x_test_batch, y_test_batch in tqdm(test_loader):
    x_test_batch = x_test_batch.to(device)#.to(float).to(device)
    y_test_pred = model(x_test_batch)
    _, y_pred_probs = torch.max(y_test_pred, dim = 1)
    all_pred.append(y_pred_probs.cpu().numpy())
    all_preds = torch.cat((all_preds.cpu(), y_pred_probs.cpu()),dim = 0)
    y_test = torch.cat((y_test, y_test_batch), dim = 0) 

In [ ]:
confusion_matrix_df = pd.DataFrame(confusion_matrix(y_test, all_pred))#.rename(columns=idx2class, index=idx2class)
sns.heatmap(confusion_matrix_df, annot=True, fmt=".2f", cmap='BuGn')
plt.xlabel("prediction")
plt.ylabel("label (ground truth)")
plt.savefig('/content/gdrive/MyDrive/Colab_Notebooks/model_charts/CMTX{}.png'.format(extension_))
plt.show()

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix
#class_names = ["Possible Dementia", "Likely Dementia", "No Dementia"]
class_vals = [0,1,2]

cr = classification_report(y_test, all_pred, class_vals, output_dict = True)
try:
    cr_file = open('/content/gdrive/MyDrive/Colab_Notebooks/model_charts/cr{}.txt'.format(extension_), 'wt')
    cr_file.write(str(cr))
    cr_file.close()
  
except:
    print("Unable to write to file")
print(classification_report(y_test, all_pred, class_vals))


# Plot ROC curve

In [ ]:
import matplotlib.pyplot as plt 
from sklearn.preprocessing import LabelBinarizer
from sklearn.metrics import roc_curve, auc, roc_auc_score


target= [0, 1, 2]

# set plot figure size
fig, c_ax = plt.subplots(1,1, figsize = (12, 8))

# function for scoring roc auc score for multi-class
def multiclass_roc_auc_score(y_test1, all_pred1, average="macro"):
    lb = LabelBinarizer()
    lb.fit(y_test1)
    y_test1 = lb.transform(y_test1)
    all_pred1 = lb.transform(all_pred1)

    for (idx, c_label) in enumerate(target):
        fpr, tpr, thresholds = roc_curve(y_test1[:,idx].astype(int), all_pred1[:,idx])
        c_ax.plot(fpr, tpr, label = '%s (AUC:%0.2f)'  % (c_label, auc(fpr, tpr)))
    c_ax.plot(fpr, fpr, 'b-', label = 'Random Guessing')
    return roc_auc_score(y_test1, all_pred1, average=average)


print('ROC AUC score:', multiclass_roc_auc_score(y_test, all_pred))

c_ax.legend()
c_ax.set_xlabel('False Positive Rate')
c_ax.set_ylabel('True Positive Rate')
plt.savefig('/content/gdrive/MyDrive/Colab_Notebooks/model_charts/roc{}.png'.format(extension_))
plt.show()

In [ ]:
# precision recall curve
from sklearn.preprocessing import label_binarize
from sklearn.metrics import precision_recall_curve


# Use label_binarize to be multi-label like settings
y = y_test.numpy()
Y = label_binarize(y, classes=[0, 1, 2])
Y_pred = label_binarize(all_pred, classes=[0, 1, 2])
n_classes = Y.shape[1]

precision = dict()
recall = dict()
for i in range(n_classes):
    precision[i], recall[i], _ = precision_recall_curve(Y[:, i],
                                                        Y_pred[:, i])
    plt.plot(recall[i], precision[i], lw=2, label='class {}'.format(i))
    
plt.xlabel("recall")
plt.ylabel("precision")
plt.legend(loc="best")
plt.title("precision vs. recall curve")
plt.savefig('/content/gdrive/MyDrive/Colab_Notebooks/model_charts/auc_pr{}.png'.format(extension_))
plt.show()

## Save the GPU CNN Model
Also includes loading on GPU

In [ ]:
# Save GPU model
model_name = 'cnn_1run.model{}'.format(extension_)
PATH = "/content/gdrive/MyDrive/Colab_Notebooks/{}".format(model_name)
torch.save(model.state_dict(), PATH)

In [ ]:
"""# Load GPU model
device = torch.device("cuda")
model = TheModelClass(*args, **kwargs)
model.load_state_dict(torch.load(PATH))
model.to(device)"""

In [ ]:
def imshow(inp, title=None):
    """Imshow for Tensor."""
    inp = inp.numpy().transpose((1, 2, 0))
    mean = np.array([0.485, 0.456, 0.406])
    std = np.array([0.229, 0.224, 0.225])
    inp = std * inp + mean
    inp = np.clip(inp, 0, 1)
    plt.imshow(inp)
    if title is not None:
        plt.title(title)
    plt.pause(0.001)  # pause a bit so that plots are updated

def visualize_model(model, num_images=6):
    was_training = model.training
    class_names = [0,1,2]
    model.eval()
    images_so_far = 0
    fig = plt.figure()

    with torch.no_grad():
        for i, (inputs, labels) in enumerate(validate_loader):
            inputs = inputs.to(device)
            labels = labels.to(device)

            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)

            for j in range(inputs.size()[0]):
                images_so_far += 1
                ax = plt.subplot(num_images//2, 2, images_so_far)
                ax.axis('off')
                if j < len(preds):
                    ax.set_title('predicted: {}'.format(class_names[preds[j]]))
                    if inputs.cpu().data[j].shape[0] == 3:  # RGB
                        imshow(inputs.cpu().data[j])
                    else:  # Grayscale
                        plt.imshow(inputs.cpu().data[j].squeeze(), cmap='gray')

                if images_so_far == num_images:
                    model.train(mode=was_training)
                    return
        model.train(mode=was_training)

visualize_model(model)  